# Properties

Data driven attribute creation

In [ ]:
import json

JSON_PATH = 'data.json'

class Record:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

    def __repr__(self):
        return f'<{self.__class__.__name__} serial={self.serial!r}>'

    @staticmethod
    def fetch(key):
        if Record.__index is None:
            Record.__index = load()
        return Record.__index[key]


class Event(Record):

    def __repr__(self):
        try:
            return f'<{self.__class__.__name__} {self.name!r}>'
        except AttributeError:
            return super().__repr__()

    @property
    def venue(self):
        key = f'venue.{self.venue_serial}'
        return self.__class__.fetch(key)

    
def load(path=JSON_PATH):
    records = {}
    with open(path) as fp:
        raw_data = json.load(fp)
    for collection, raw_records in raw_data['Schedule'].items():
        record_type = collection[:-1]
        cls_name = record_type.capitalize()
        cls = globals().get(cls_name, Record)
        if inspect.isclasss(cls) and issubclass(cls, Record):
            factory = cls
        else:
            factory = Record
        for raw_record in raw_records:
            key = f'{record_type}.{raw_record["serial"]}'
            records[key] = factory(**raw_record)
    return records

Property to retrieve a linked record

Property overriding an existing attribute

### Caching properties with functools

### Using a property for attribute validation

We can use a property in place of a getter and a setter without changing the class interface.

In [12]:
class LineItem:
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price

    def subtotal(self):
        return self.weight * self.price
    
    @property
    def weight(self):
        return self.__weight
    
    @weight.setter
    def weight(self, value):
        if value > 0:
            self.__weigth = value
        else:
            raise ValueError('value must be > 0')
        
walnuts = LineItem('walnuts', 0, 10)

ValueError: value must be > 0

### A proper look at properties

When an instance and its class both have a data attribute by the same name, the instance attribute shadows the class attribute, at least when reading it through that instance

In [2]:
# Example of shadowing

class Class:
    data = 'the class data attr'
    

obj = Class()
print(vars(obj))
print(obj.data)
obj.data = 'bar'
print(vars(obj))
print(obj.data)
print(Class.data)

Class.data = property(lambda self: 'the data prop value')
print(obj.data)
del Class.data
print(obj.data)
print(Class.data)

{}
the class data attr
{'data': 'bar'}
bar
the class data attr
the data prop value
bar


AttributeError: type object 'Class' has no attribute 'data'

Properties override instance attributes

In [ ]:
class Class:
    @property
    def prop(self):
        return 'the prop value'
    
print(Class.prop)
obj = Class()
print(obj.prop)


the prop value


In [2]:
obj.prop = 'foo'

AttributeError: can't set attribute 'prop'

In [3]:
print(vars(obj))
obj.__dict__['prop'] = 'foo'
print(vars(obj))
print(obj.prop)
Class.prop = 'baz'
print(obj.prop)

{}
{'prop': 'foo'}
the prop value
foo


### Property factory

In [ ]:
def quantity(storage_name):
    def qty_getter(instance):
        return instance.__dict__[storage_name]
    
    def qty_setter(instance, value):
        if value > 0:
            instance.__dict__[storage_name] = value
        else:
            raise ValueError('value must be > 0')
    
    return property(qty_getter, qty_setter)


class LineItem:
    weight = quantity('weight')
    price = quantity('price')

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price

    def subtotal(self):
        return self.weight*self.price
    
nutmeg = LineItem('Moluccan nutmeg', 8, 13.95)
print(nutmeg.weight, nutmeg.price)
print(nutmeg.__dict__)


8 13.95
{'description': 'Moluccan nutmeg', 'weight': 8, 'price': 13.95}


### Handling attribute deletion
The `del` keyword can be used to delete not only variables but also attributes

In [4]:
class Demo:
    pass

d = Demo()
d.color = 'green'
print(d.color)
del d.color
print(d.color)

green


AttributeError: 'Demo' object has no attribute 'color'